In [1]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 3.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [1]:
# MLP for the IMDB problem
import numpy as np
import keras
import pickle 
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Average
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.preprocessing import sequence

import matplotlib.pyplot as plt
%matplotlib inline
import warnings,logging, os
warnings.filterwarnings('ignore')
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.insert(0, 'drive/My Drive/WordEmbed workshop/sentiment_analysis')

from utils import *

In [4]:
# load the dataset but only keep the top n words, zero the rest
top_words = 15000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
imdb_words = imdb.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [0]:
INDEX_FROM=3
MAX_NB_WORDS = 15000
embed_dim = 300
max_words = 500

word_to_id = {k:(v+INDEX_FROM) for k,v in imdb_words.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
nb_words = min(MAX_NB_WORDS, len(word_to_id))
id_to_word = {value:key for key,value in word_to_id.items()}

In [6]:
[id_to_word[i] for i in X_train[0]]

['<START>',
 'this',
 'film',
 'was',
 'just',
 'brilliant',
 'casting',
 'location',
 'scenery',
 'story',
 'direction',
 "everyone's",
 'really',
 'suited',
 'the',
 'part',
 'they',
 'played',
 'and',
 'you',
 'could',
 'just',
 'imagine',
 'being',
 'there',
 'robert',
 '<UNK>',
 'is',
 'an',
 'amazing',
 'actor',
 'and',
 'now',
 'the',
 'same',
 'being',
 'director',
 '<UNK>',
 'father',
 'came',
 'from',
 'the',
 'same',
 'scottish',
 'island',
 'as',
 'myself',
 'so',
 'i',
 'loved',
 'the',
 'fact',
 'there',
 'was',
 'a',
 'real',
 'connection',
 'with',
 'this',
 'film',
 'the',
 'witty',
 'remarks',
 'throughout',
 'the',
 'film',
 'were',
 'great',
 'it',
 'was',
 'just',
 'brilliant',
 'so',
 'much',
 'that',
 'i',
 'bought',
 'the',
 'film',
 'as',
 'soon',
 'as',
 'it',
 'was',
 'released',
 'for',
 '<UNK>',
 'and',
 'would',
 'recommend',
 'it',
 'to',
 'everyone',
 'to',
 'watch',
 'and',
 'the',
 'fly',
 'fishing',
 'was',
 'amazing',
 'really',
 'cried',
 'at',
 'th

In [0]:
# maximum words for a review. We truncate longer reviews and pad shorter reviews to make the length 500
max_words = 500 
X_train = sequence.pad_sequences(X_train, maxlen=max_words, padding='post', truncating='post', value=0.0)
X_test = sequence.pad_sequences(X_test, maxlen=max_words, padding='post', truncating='post', value=0.0)

In [8]:
# This is with random initializations for embeddings
model = Sequential()
model.add(Embedding(top_words, 300, input_length=max_words))
model.add(keras.layers.Lambda(lambda x: keras.backend.mean(x, axis=1)))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 300)          4500000   
_________________________________________________________________
lambda_1 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               75250     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 4,575,501
Trainable params: 4,575,501
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
 - 36s - loss: 0.5039 - acc: 0.7503 - val_loss: 0.3340 - val_acc: 0.8586
Epoch 2/2
 - 35s - loss: 0.2433 - acc: 0.9061 - val_loss: 0.3075 - val_acc: 0.8736
Accuracy: 87.36%


Below snippet loads the model. Would take a couple of minutes.

In [0]:
# #embedding matrix
# MAX_NB_WORDS = 15000
# embed_dim = 300
# max_words = 500
# words_not_found = []
# nb_words = min(MAX_NB_WORDS, len(word_to_id))
# embedding_matrix = np.zeros((nb_words, embed_dim))
# for word, i in word_to_id.items():
#     if i >= nb_words:
#         continue
#     embedding_vector = embeddings_index[word]
#     if (embedding_vector is not None) and len(embedding_vector) > 0:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
#     else:
#         words_not_found.append(word)

In [0]:
embedding_matrix = load_obj('drive/My Drive/WordEmbed workshop/sentiment_analysis/embedding_matrix')

In [10]:
# This model uses pretrained embeddings, but does not train the embeddings while the model is training.
model2 = Sequential()
model2.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_words, trainable=False))
model2.add(keras.layers.Lambda(lambda x: keras.backend.mean(x, axis=1)))
model2.add(Dense(250, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

# Fit the model
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores2 = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores2[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 300)          4500000   
_________________________________________________________________
lambda_2 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               75250     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 4,575,501
Trainable params: 75,501
Non-trainable params: 4,500,000
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
 - 14s - loss: 0.6747 - acc: 0.5824 - val_loss: 0.6534 - val_acc: 0.6148
Epoch 2/2
 - 14s - loss: 0.6205 - acc: 0.6743 - val_loss: 0.5987 - val_acc: 0.7098
Accuracy: 70.98%


In [11]:
# This model uses trainable pretrained word embeddings
model3 = Sequential()
model3.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_words, trainable=True))
model3.add(keras.layers.Lambda(lambda x: keras.backend.mean(x, axis=1)))
model3.add(Dense(250, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model3.summary())
# Fit the model
model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores3 = model3.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores3[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 300)          4500000   
_________________________________________________________________
lambda_3 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               75250     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 251       
Total params: 4,575,501
Trainable params: 4,575,501
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
 - 37s - loss: 0.5225 - acc: 0.7389 - val_loss: 0.3557 - val_acc: 0.8540
Epoch 2/2
 - 36s - loss: 0.2575 - acc: 0.9006 - val_loss: 0.2794 - val_acc: 0.8871
Accuracy: 88.71%
